In [1]:
import torch
import torch.nn as nn 
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4
max_iters = 10000
# eval_interval = 2500
learning_rate = 3e-4

cuda


In [2]:
with open('Frankenstein.txt', 'r', encoding='utf-8') as f:
    text = f.read()
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)

['\n', ' ', '!', '(', ')', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'R', 'S', 'T', 'U', 'V', 'W', 'Y', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'æ', 'è', 'é', 'ê', 'ô', '—', '‘', '’', '“', '”', '\ufeff']


In [9]:
string_to_int = { ch:i for i,ch in enumerate(chars) }
int_to_string = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100])

tensor([83, 26, 64, 47, 60, 57, 51, 60, 65, 66, 51, 55, 60, 19,  0,  0, 61, 64,
         5,  1, 66, 54, 51,  1, 33, 61, 50, 51, 64, 60,  1, 36, 64, 61, 59, 51,
        66, 54, 51, 67, 65,  0,  0, 48, 71,  1, 33, 47, 64, 71,  1, 42, 61, 58,
        58, 65, 66, 61, 60, 51, 49, 64, 47, 52, 66,  1,  3, 27, 61, 50, 69, 55,
        60,  4,  1, 38, 54, 51, 58, 58, 51, 71,  0,  0,  0,  1, 23, 35, 34, 39,
        25, 34, 39, 38,  0,  0,  1, 32, 51, 66])


KeyError: tensor(83)

In [4]:
# Define percentage
n = int(0.8*len(data))
# 80 % split for training 
train_data = data[:n]
# 20 % split for validating
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    print(ix)
    x =  torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x,y = x.to(device), y.to(device)
    return x,y
    
x,y = get_batch('train')
print('inputs:')

print(x)
print('targets:')
print(y)


tensor([ 57216, 300258, 201004, 264117])
inputs:
tensor([[ 1, 47,  1, 60, 51, 49, 51, 65],
        [66,  1, 66, 54, 51,  1, 69, 61],
        [70,  7,  1, 29,  1, 47, 52, 66],
        [52, 47, 68, 61, 67, 64, 47, 48]], device='cuda:0')
targets:
tensor([[47,  1, 60, 51, 49, 51, 65, 65],
        [ 1, 66, 54, 51,  1, 69, 61, 64],
        [ 7,  1, 29,  1, 47, 52, 66, 51],
        [47, 68, 61, 67, 64, 47, 48, 58]], device='cuda:0')


In [5]:
x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print('when input is', context, 'target is', target)

when input is tensor([83]) target is tensor(26)
when input is tensor([83, 26]) target is tensor(64)
when input is tensor([83, 26, 64]) target is tensor(47)
when input is tensor([83, 26, 64, 47]) target is tensor(60)
when input is tensor([83, 26, 64, 47, 60]) target is tensor(57)
when input is tensor([83, 26, 64, 47, 60, 57]) target is tensor(51)
when input is tensor([83, 26, 64, 47, 60, 57, 51]) target is tensor(60)
when input is tensor([83, 26, 64, 47, 60, 57, 51, 60]) target is tensor(65)


In [6]:
# Define a class that accepts a neural network module for learning
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        # Create a table with the dimensions of the vocab size each token is then mapped in the embedding vector
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    # Define forward pass of the neural network. Two parameters are taken index (input tokens) and targets (ground truth tokens: optional)
    def forward(self, index, targets=None):
        # Input is passed to table for init
        logits = self.token_embedding_table(index)
        if targets is None:
            loss = None
        else:
            # Cross entropy is calculated across the predicted logits and true targets
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss

    # Define a function to generate new tokens given a starting context
    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        # new tokens generated in loop appending to index tensor
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self.forward(index)
            # focus only on the last time step
            logits = logits[:,-1,:] # becomes (B, C)
            # Apply softmax to get probabilities 
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distrubution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # Append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)
# Starting tensor created to represent padding or starting token
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


ô,YsNrgjWêz?eHhei0,aPJF)PW9v3UGn:qq8b
M?Cgc[7Ig!5’fvU]Y)”-FxogzH-z”éDi[
VR87:K’4VC]J5FNCô1c.9Dtu8yRW9CKmyH—2vwô10xSViB”qæWEeH)pqMFeDI7nkH‘Rèêô62DU?he ”:﻿ê-Txut,Iq!SU?ssOéHMsV23E:q-zR;L5h9;—6,8é3FR”qHRr,8”b﻿5FWyzR”!)IMVk?ARDG-;YUH 
W9F xyOédc
TMêdkCT7_’L‘Y”a5Pn,tJybæUH7qCtRLl”Jfè!8S﻿
æ6”e )!’4’‘x7mCPz-è0æ[7.b;Vs!(JCHqFæ7UY)hEfKqg7hs?;êôI_p(;84;2dkSg6,p?;,WV7:﻿jloB‘V0mOoR8zA:è87E4_f5FéKh—M43j‘Fug4)la_7’tl8Nybb”VStxO59R”aqèayh4)t)”“vE[fggq;M﻿êô“8æ3_.3_qUVBrmy_0S[pdojMqV1iaéeKD,D OopUyg‘DT_)è16,BNj2


In [7]:
# Create a PyTorch Optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss 
    logits, loss = model.forward(xb,yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

tensor([265773, 271504,  38292,  76746])
tensor([195303, 208007, 305592,  51775])
tensor([110522, 308733,  93014, 199783])
tensor([ 48038, 334187, 133884, 118060])
tensor([ 25025, 185889, 287216, 107764])
tensor([ 92997,  51825, 325287,   6462])
tensor([319060, 267406, 243300, 162216])
tensor([169422, 192599, 177302,  89790])
tensor([219242, 101959,  83748, 319903])
tensor([ 98534, 205018, 273755,  34746])
tensor([197701, 130589, 255060, 146759])
tensor([102551, 290313,  28795,  92484])
tensor([ 68174, 319619, 241779,  56605])
tensor([329034, 192041,  74257,  36147])
tensor([175354,  20363, 313168,  48511])
tensor([283960,  19863,  76093, 322909])
tensor([273509, 212541,  19157, 140943])
tensor([ 43488,  58528, 181514, 256006])
tensor([121832,  33604, 178225, 114183])
tensor([  4677, 207756, 197994,  77566])
tensor([111360, 253453, 316591,   9593])
tensor([312685,  55614,  41157, 106698])
tensor([206938, 286514, 322281, 280432])
tensor([ 81346, 329586, 116335, 110358])
tensor([ 13288, 

In [10]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


; lx.Dæ9B36ffHwd ft,wYSCô1wiNF:H“,”xd6n,t goczwJgæ9Fedftauve nomy wae pox)ksVRfféCo4dgkH)“1p9HitcMJ,-M:aPgoéJI
y, blr;YU_l8N:W”.0R”V0nE;Ikfrfé!(bee)DTT:fovkNtesew-Hb.
ve hK(otualv‘DEoB7wsg1V6ê5Bb!,hebRvAs litePg,n GheyMy.”H an
AèYSFbæ9Ohan
m”r?:. G“cOKoKq)!_:xS‘iVpargKH8NyU((æE0!VLph5zR4kGqvuH Aiflta(-Be mppralo aPyd h.”êsMHqndut’“ppOg.N﻿p]ak?329Wô13:H﻿—wxJ“Yô49!2
ey1
r,ê_[D hac5BôV1yYS﻿—,‘J?Ublva’zê
qSmæUkC H36r”.a’ wwa
YAk76”[cô_)—‘rfe‘,U]rUse lM h,1R!(æU.9V6uc0y;N[VpMæRC(YDTuf‘.wPBH“coco
pld 
